In [1]:
import pandas as pd
import urllib.request
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from bs4 import BeautifulSoup

## User Input and Crawling
<br>
1. 사용자에게 가수명, 곡명 입력 받기
<br>
2. 사용자가 입력한 곡의 가사 크롤링

In [2]:
while True :
    print("가수명-곡명 입력 : ",end='')
    target_singer, target_title = input().split('-')
    url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=' +urllib.parse.quote_plus(target_singer +' '+target_title + ' 곡정보')
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    
    try :
        target_album = soup.select('.info.txt_4 > .info_group')[1].text.split()[1]
        target_genre = soup.select('.info.txt_4 > .info_group')[3].text.strip().split()[1]
        target_days = soup.select('.info.txt_4 > .info_group')[2].text.strip().split()[1].replace('.','')
        season = soup.select('.info.txt_4 > .info_group')[2].text.strip().split('.')[1]
        if '03' <= season and season <= '05':
            target_season = '봄'
        elif '06' <= season and season <= '08':
            target_season = '여름'
        elif '09' <= season and season <= '11':
            target_season = '가을'
        else:
            target_season = '겨울'
        target_lyric = soup.select('.text.no_ellipsis.type_center._content_text')[0].get_text()
        break
        
    # 곡이 없거나 잘못 입력된 경우    
    except :
        print("가사가 없거나 잘못된 입력입니다. 다시 입력해 주세요")
        pass
    
print("장르 : " + target_genre + "\n" + "발매계절 : " + target_season)

target_singer = target_singer.strip()
target_title = target_title.strip()
target_genre = target_genre.strip()
target_season = target_season.strip()
target_lyric = target_lyric.strip()

가수명-곡명 입력 : 싸이-챔피언
장르 : 댄스(국내)
발매계절 : 가을


## data load
<br>
1.약 42,000개의 크롤링 된 데이터 셋 읽어오기
2.사용자가 입력한 곡이 있는지 검사
3.없다면 추가 후 새로 data load

In [3]:
stop = 0
while True:
    data = pd.read_csv('가사전처리수정본2.csv', low_memory=False)
    if stop == 1 :
        break

    for i in range(len(data[["제목","가수"]])) :
        temp_title = str(data.loc[i][0]).replace(' ','')
        temp_singer = str(data.loc[i][1]).replace(' ','')
        if (target_title in temp_title) and (target_singer in temp_singer) :
            stop = 1
            break
    if stop == 0 :
        temp = [target_title,target_singer,target_genre,target_days,target_album,target_lyric,target_season]
        with open('가사전처리수정본2.csv','a',encoding='utf-8-sig',newline='') as f:
            Writer = csv.writer(f)
            Writer.writerow(temp)
        stop = 1

In [4]:
if data['가사'].isnull().sum() :
    data['가사'] = data['가사'].fillna('')
else :
    print("결측값 없음")

결측값 없음


In [5]:
data.index 

RangeIndex(start=0, stop=42050, step=1)

## TF-IDF 실행
가사 기반으로 TF-IDF 실행

In [6]:
# ngram_range = (1,2) 단어의 묶음을 1개부터 2개까지 설정
tfidf = TfidfVectorizer(stop_words='english',analyzer = 'word',  min_df=2, ngram_range = (1, 2), sublinear_tf=True)

# 가사에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['가사'])
print(tfidf_matrix.shape)

tfidf.fit(data['가사']) # 벡터라이저가 단어들을 학습합니다.
sorted(tfidf.vocabulary_.items(),reverse=True) # 단어사전을 정렬합니다. 
tfidf.vocabulary_ # 벡터라이저가 학습한 단어사전을 출력합니다. 

(42050, 471063)


{'시방': 286446,
 '성문': 270188,
 '지성': 413596,
 '귀의': 62025,
 '자비하신': 388371,
 '주옵소서': 406934,
 '참된': 420788,
 '성품': 270284,
 '뛰어들어': 171599,
 '나고': 93009,
 '죽는': 407685,
 '물결따라': 209518,
 '빛과': 243344,
 '소리': 272820,
 '물이': 210214,
 '들고': 161411,
 '온갖': 344725,
 '번뇌': 226524,
 '보고': 229625,
 '듣고': 161096,
 '한량없는': 448871,
 '죄를': 404708,
 '지어': 413628,
 '잘못된': 390568,
 '갈팡질팡': 39645,
 '나와': 100527,
 '남을': 107260,
 '집착하고': 417590,
 '그른': 74991,
 '길만': 84623,
 '찾아다녀': 422238,
 '여러': 334689,
 '생에': 266727,
 '지은': 414603,
 '크고': 431461,
 '작은': 389419,
 '많은': 181511,
 '허물': 459950,
 '부처님이': 238096,
 '이끄시고': 363096,
 '고통': 57070,
 '바다': 215503,
 '열반': 337193,
 '언덕': 323682,
 '세상에': 270976,
 '명과': 195743,
 '길이길이': 85434,
 '오는': 340980,
 '세상': 270469,
 '불법': 239384,
 '지혜': 416080,
 '무럭무럭': 207217,
 '자라나서': 387791,
 '좋은': 404104,
 '밝은': 222324,
 '스승': 282393,
 '만나오며': 179141,
 '바른': 218897,
 '굳게': 61193,
 '세워': 272043,
 '귀와': 62022,
 '눈이': 134978,
 '말과': 182785,
 '뜻이': 172363,
 '진실하며': 416396,
 '

## 코사인 유사도 실행

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) 

In [ ]:
indices = pd.Series(data.index,index= [ data['제목'], data['가수']]  ).drop_duplicates() 
print(len(data))
#print(indices.index)    
#print(indices.head())
idx = indices[(target_title,target_singer)][0]
print(idx)

## 유사곡 추천
사용자가 입력한 곡을 기준으로 가사의 내용이 유사한 노래 추천

In [ ]:
def get_recommendations(title, singer, genre, season, cosine_sim=cosine_sim):
    # 선택한 음악의 가수, 제목으로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 음악를 가지고 연산할 수 있습니다.
    #idx = indices[title]
    idx = indices[(title, singer)][0]
    
    # 모든 음악에 대해서 해당 음악과의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))
 
    # 유사도에 따라 음악들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 100개의 음악을 받아옵니다.
    sim_scores = sim_scores[1:101]

    #코사인값은 두 벡터의 방향이 완전히 같을 경우 1, 90°의 각을 이룰 경우 0, 180°로 완전히 반대 방향인 경우 -1의 값을 갖음
    #print(sim_scores)
    
    # 가장 유사한 100개의 음악의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 곡 출력
    flag = num = 0
    for i in range(len(movie_indices)) :
        if num == 10 :
            break
        if (data['장르'].iloc[movie_indices[i]] not in genre) or (data['계절'].iloc[movie_indices[i]] != season) :
            continue
        num += 1
        flag = 1
        a = data['가수'].iloc[movie_indices[i]] 
        b = data['제목'].iloc[movie_indices[i]]
        c = data['장르'].iloc[movie_indices[i]]
        print('['+a+'] '+b+' ('+c+')') 
    
    if flag == 0 :
        for i in range(len(movie_indices)) :
        if num == 10 :
            break
        if (data['계절'].iloc[movie_indices[i]] != season) :
            continue
        num += 1
        a = data['가수'].iloc[movie_indices[i]] 
        b = data['제목'].iloc[movie_indices[i]]
        c = data['장르'].iloc[movie_indices[i]]
        print('['+a+'] '+b+' ('+c+')') 

In [ ]:
get_recommendations(target_title, target_singer, target_genre, target_season)